# Obtaining Flight Data for Top Destinations #

## Overview
This notebook combines functionality to:
1. Filter destinations based on weather conditions similar to Greece (±2°C temperature, equal or lower rainfall)
2. Sort remaining destinations by tourism levels to find quieter travel options
3. Use SkyScanner API to retrieve flight information for the top destinations

## Workflow
1. Load and process weather data from `Completed_Weather_Data_For_Destinations.csv`
2. Filter destinations with similar climate to Greece
3. Sort by tourism levels to identify quieter alternatives
4. Use SkyScanner API to retrieve location codes and IDs
5. Search for flight options from selected origin to top destinations
6. Export relevant flight data to Excel for further analysis

In [ ]:
!pip install requests # Ensure requests installed to call the API

In [ ]:
!pip install pandas openpyxl  # to write results to a spreadsheet

In [1]:
import pandas as pd # To use pandas library for cleaning, transforming and analysing data
import requests # To call API
import json # To format API responses
from datetime import datetime # To use datetime formatting of arrival and departure of flights
from pathlib import Path # all the data will be stored and looked for in the right place.

In [ ]:
RAPIDAPI_KEY = '******************************' # Enter your API key
RAPIDAPI_HOST = '****************************' # Enter your API host name

In [ ]:
# Need the code for origin and destination in order to search flights.
# Therefore, using the auto-complete endpoint of SkyScanner API to retrieve the code using place name to query.

def find_code(place):

    url = 'https://skyscanner89.p.rapidapi.com/flights/auto-complete'
    
    headers = {
    	'x-rapidapi-key': RAPIDAPI_KEY,    
    	'x-rapidapi-host': RAPIDAPI_HOST    
    }

    query = {'query': place}

    response = requests.get(url, headers=headers, params = query)

    if response.status_code == 200:

        result = response.json()
    
        suggested = result['inputSuggest']

        if suggested:
            choice = suggested[0]['navigation']['relevantFlightParams']
            name = choice['localizedName']
            code = choice['skyId']
            _id = choice['entityId']
            found_code = {'Name': name, 'Code': code, 'ID': _id}
           # print(name, code, _id) to check 
    
        return found_code

    else:
        print('Failed to retrieve data: ', response.status_code)
        return response.text

In [ ]:
# checking function returns expected results
# find_code('Majorca')

In [ ]:
# checking
# find_code('Manchester')['Code']

In [ ]:
# checking
# find_code('London')

In [ ]:
# Writing function to retrieve flights.

def find_flights(date, origin, origin_id, destination, destination_id):

    url = 'https://skyscanner89.p.rapidapi.com/flights/one-way/list'

    headers = {
    	'x-rapidapi-key': RAPIDAPI_KEY,      
    	'x-rapidapi-host': RAPIDAPI_HOST
    }
    
    query = {
        'date': date,
        'origin': origin,
        'originId': origin_id,
        'destination': destination,
        'destinationId': destination_id,
        'adults':'1',
        'locale':'en-GB'
    }
    
    response = requests.get(url, headers=headers, params=query)

    if response.status_code == 200:
        flights = response.json()
        search_results = flights['data']['itineraries']['buckets']
        return search_results

    else:
        print('Failed to retrieve data: ', response.status_code)
        return response.text


### Testing find_flights function ###

In [ ]:
# Testing function
# date = '2025-06-06'
# origin = find_code('Manchester')['Code']
# origin_id = find_code('Manchester')['ID']
# destination = find_code('Majorca')['Code']
# destination_id = find_code('Majorca')['ID']

# man_to_maj = find_flights(date, origin, origin_id, destination, destination_id)

### Testing appending relevant results to a list, to convert to a pandas dataframe ###

In [ ]:
# found_flights = [] # Using a list to collect the dictionaries, for easy conversion to pandas dataframe.

# for result in man_to_maj:
#     details = result['items'] #This is a list of dictionaries, so need to iterate through again

#     for flight in details:
        
#         leg = flight.get('legs', [{}])[0] #These are each a list of length 1 and so need index [0]
#         segment = leg.get('segments', [{}])[0]

#         # These find the relevant info using the dictionary keys, .get used to avoid errors and provide defaults.
    
#         flight_id = flight.get('id', 'N/A')
#         carrier_name = segment.get('marketingCarrier', {}).get('name', 'N/A')
#         flight_number = segment.get('flightNumber', 'N/A')
#         origin = leg.get('origin', {}).get('name', 'N/A')
#         departure_dandt = leg.get('departure', '')
#         destination = leg.get('destination', {}).get('name', 'N/A')
#         arrival_dandt = leg.get('arrival', '')
#         duration = leg.get('durationInMinutes', 'N/A')
#         stops = leg.get('stopCount', 0)
#         price = flight.get('price', {}).get('raw', 'N/A')

        ## Arrival and departure date and time need splitting up, except where they are unavailable

        # try:
        #     departure_dt = datetime.fromisoformat(departure_dandt)
        #     departure_date = departure_dt.strftime('%Y-%m-%d')
        #     departure_time = departure_dt.strftime('%H:%M')
        # except:
        #     departure_date = departure_time = 'N/A'
        
        # try:
        #     arrival_dt = datetime.fromisoformat(arrival_dandt)
        #     arrival_date = arrival_dt.strftime('%Y-%m-%d')
        #     arrival_time = arrival_dt.strftime('%H:%M')
        # except:
        #     arrival_date = arrival_time = 'N/A'
    
        # relevant_data = {'ID': flight_id,
        #                  'Airline': carrier_name,
        #                  'Flight no.': flight_number,
        #                  'Origin': origin,
        #                  'Departure': departure_dandt,
        #                  'Departure date': departure_date,
        #                  'Departure time': departure_time,
        #                  'Destination': destination,
        #                  'Arrival': arrival_dandt,
        #                  'Arrival date': arrival_date,
        #                  'Arrival time': arrival_time,
        #                  'Duration': duration,
        #                  'Stops': stops,
        #                  'Price' : price}

        # found_flights.append(relevant_data)

In [ ]:
# print(found_flights) # Checking it works

In [ ]:
# df = pd.DataFrame(found_flights)

### Testing writing data to a csv file ###

In [ ]:
# data_dir = Path('project_data')

# excel_file_path = data_dir / 'man_to_maj_flights.xlsx'

# df.to_excel(excel_file_path, index=False)

# man_maj_sheet = pd.read_excel(excel_file_path)

In [ ]:
# man_maj_sheet #Reading the xlsx results

### Testing wrapping the above process into one function ###

In [ ]:
# def search_flights(from_here, to_there):

#     # Getting parameters

#     date = '2025-06-06'
#     origin = find_code(from_here)['Code']
#     origin_id = find_code(from_here)['ID']
#     destination = find_code(to_there)['Code']
#     destination_id = find_code(to_there)['ID']

#     from_here_to_there = find_flights(date, origin, origin_id, destination, destination_id)

#     found_flights = [] # Using a list to collect the dictionaries, for easy conversion to pandas dataframe.

#     for result in from_here_to_there:
#         details = result['items'] #This is a list of dictionaries, so need to iterate through again
    
#         for flight in details:
            
#             leg = flight.get('legs', [{}])[0] #These are each a list of length 1 and so need index [0]
#             segment = leg.get('segments', [{}])[0]
    
#             # These find the relevant info using the dictionary keys, .get used to avoid errors and provide defaults.
        
#             flight_id = flight.get('id', 'N/A')
#             carrier_name = segment.get('marketingCarrier', {}).get('name', 'N/A')
#             flight_number = segment.get('flightNumber', 'N/A')
#             origin = leg.get('origin', {}).get('name', 'N/A')
#             departure_dandt = leg.get('departure', '')
#             destination = leg.get('destination', {}).get('name', 'N/A')
#             arrival_dandt = leg.get('arrival', '')
            # duration = leg.get('durationInMinutes', 'N/A')
            # stops = leg.get('stopCount', 0)
            # price = flight.get('price', {}).get('raw', 'N/A')
    
            # # Arrival and departure date and time need splitting up, except where they are unavailable
    
            # try:
            #     departure_dt = datetime.fromisoformat(departure_dandt)
            #     departure_date = departure_dt.strftime('%Y-%m-%d')
            #     departure_time = departure_dt.strftime('%H:%M')
            # except:
            #     departure_date = departure_time = 'N/A'
            
            # try:
            #     arrival_dt = datetime.fromisoformat(arrival_dandt)
            #     arrival_date = arrival_dt.strftime('%Y-%m-%d')
            #     arrival_time = arrival_dt.strftime('%H:%M')
            # except:
            #     arrival_date = arrival_time = 'N/A'

            # relevant_data = {'ID': flight_id,
            #                  'Airline': carrier_name,
            #                  'Flight no.': flight_number,
            #                  'Origin': origin,
            #                  'Departure': departure_dandt,
            #                  'Departure date': departure_date,
            #                  'Departure time': departure_time,
            #                  'Destination': destination,
            #                  'Arrival': arrival_dandt,
            #                  'Arrival date': arrival_date,
            #                  'Arrival time': arrival_time,
            #                  'Duration': duration,
            #                  'Stops': stops,
            #                  'Price' : price}
    
        #     found_flights.append(relevant_data)

        # df = pd.DataFrame(found_flights)

        # data_dir = Path('project_data')

        # excel_file_path = data_dir / f'{from_here}_{to_there}_flights.xlsx'

        # df.to_excel(excel_file_path, index=False)

        # flights_sheet = pd.read_excel(excel_file_path)

        # return flights_sheet

In [ ]:
# search_flights('Manchester', 'Crete') # Checking it works 

## Using the above function to gather relevant flight data for the top three destinations ##

The following was also displayed in notebook five. It has been included here as it is relevant to obtaining flight data.

In [2]:
df = pd.read_csv('project_data/Complete_Weather_Data_For_Destinations.csv')

df.head()

,Unnamed: 0,Country,Arrivals,Units,2018,2019,2020,2021,2022,Mean Total Arrivals,Recommendation,latitude,longitude,Avg Temp(°C),Avg Rainfall(mm)
0,0,Albania,total arrivals,thousands,5927.0,6406.0,2658.0,5689.0,7543.8,5644.76,Go,41.153332,20.168331,28.0,1.8
1,1,Algeria,total arrivals,thousands,2657.0,2371.0,591.0,125.0,1398.0,1428.40,Go,28.033886,1.659626,34.0,0.9
2,2,American Samoa,total arrivals,thousands,51.8,58.6,0.9,NaN,NaN,37.10,No Go,-14.270972,-170.132217,0.0,0.0
3,3,Andorra,total arrivals,thousands,8328.0,8235.0,5207.0,5422.0,8426.7,7123.74,Go,42.546245,1.601554,28.0,0.7
4,4,Angola,total arrivals,thousands,218.0,218.0,64.0,64.0,130.0,138.80,No Go,-11.202692,17.873887,0.0,0.0


In [ ]:
greece_df = df[df['Country'] == 'Greece']

In [ ]:
greece_df.head()

In [ ]:
greece_data = df[df['Country'] == 'Greece'].iloc[0]

In [ ]:
greece_temp = greece_data['Avg Temp(°C)']
greece_rain = greece_data['Avg Rainfall(mm)']

In [ ]:
# Filtering the countries based on temperature and rainfall in comparison to Greece
top_destinations_df = df[
    (df['Avg Temp(°C)'] >= greece_temp - 2) &
    (df['Avg Temp(°C)'] <= greece_temp + 2) &
    (df['Avg Rainfall(mm)'] <= greece_rain)
]

In [ ]:
top_destinations_df.head()

In [ ]:
# Ordering the 5 countries from top_destinations_df from lowest total arrivals to highest
ordered_top_destinations = top_destinations_df.sort_values('Mean Total Arrivals').reset_index(drop=True)

In [ ]:
ordered_top_destinations.head()

In [ ]:
# Testing function to find airport of capital of Aruba
find_code('Oranjestad') 

In [ ]:
# Testing function to find airport of capital of Azerbaijan
find_code('Baku')

In [ ]:
# Testing function to find airport of capital of Australia
find_code('Canberra')

In [ ]:
# Testing function to find airport of capital of Greece
find_code('Athens')

In [ ]:
date = '2025-06-06'
origin = find_code('Manchester')['Code']
origin_id = find_code('Manchester')['ID']
destination = find_code('Athens')['Code']
destination_id = find_code('Athens')['ID']

man_to_athens = find_flights(date, origin, origin_id, destination, destination_id)

In [ ]:
data_dir = Path('project_data')

excel_file_path = data_dir / 'man_to_athens_flights.xlsx'

df.to_excel(excel_file_path, index=False)

man_athens_sheet = pd.read_excel(excel_file_path)

In [ ]:
def search_flights(from_here, to_there):

    # Getting parameters

    date = '2025-06-06'
    origin = find_code(from_here)['Code']
    origin_id = find_code(from_here)['ID']
    destination = find_code(to_there)['Code']
    destination_id = find_code(to_there)['ID']

    from_here_to_there = find_flights(date, origin, origin_id, destination, destination_id)

    found_flights = [] # Using a list to collect the dictionaries, for easy conversion to pandas dataframe.

    for result in from_here_to_there:
        details = result['items'] #This is a list of dictionaries, so need to iterate through again
    
        for flight in details:
            
            leg = flight.get('legs', [{}])[0] #These are each a list of length 1 and so need index [0]
            segment = leg.get('segments', [{}])[0]
    
            # These find the relevant info using the dictionary keys, .get used to avoid errors and provide defaults.
        
            flight_id = flight.get('id', 'N/A')
            carrier_name = segment.get('marketingCarrier', {}).get('name', 'N/A')
            flight_number = segment.get('flightNumber', 'N/A')
            origin = leg.get('origin', {}).get('name', 'N/A')
            departure_dandt = leg.get('departure', '')
            destination = leg.get('destination', {}).get('name', 'N/A')
            arrival_dandt = leg.get('arrival', '')
            duration = leg.get('durationInMinutes', 'N/A')
            stops = leg.get('stopCount', 0)
            price = flight.get('price', {}).get('raw', 'N/A')
    
            # Arrival and departure date and time need splitting up, except where they are unavailable
    
            try:
                departure_dt = datetime.fromisoformat(departure_dandt)
                departure_date = departure_dt.strftime('%Y-%m-%d')
                departure_time = departure_dt.strftime('%H:%M')
            except:
                departure_date = departure_time = 'N/A'
            
            try:
                arrival_dt = datetime.fromisoformat(arrival_dandt)
                arrival_date = arrival_dt.strftime('%Y-%m-%d')
                arrival_time = arrival_dt.strftime('%H:%M')
            except:
                arrival_date = arrival_time = 'N/A'

            relevant_data = {'ID': flight_id,
                             'Airline': carrier_name,
                             'Flight no.': flight_number,
                             'Origin': origin,
                             'Departure': departure_dandt,
                             'Departure date': departure_date,
                             'Departure time': departure_time,
                             'Destination': destination,
                             'Arrival': arrival_dandt,
                             'Arrival date': arrival_date,
                             'Arrival time': arrival_time,
                             'Duration': duration,
                             'Stops': stops,
                             'Price' : price}
    
            found_flights.append(relevant_data)

        df = pd.DataFrame(found_flights)

        data_dir = Path('project_data')

        excel_file_path = data_dir / f'{from_here}_{to_there}_flights.xlsx'

        df.to_excel(excel_file_path, index=False)

        flights_sheet = pd.read_excel(excel_file_path)

        return flights_sheet

In [ ]:
search_flights('Manchester', 'Oranjestad')

In [ ]:
search_flights('Manchester', 'Baku')

In [ ]:
search_flights('Manchester', 'Canberra')

In [ ]:
search_flights('Manchester', 'Athens')

### Finding the minimum and average price of each flight, and the minimum and average flight duration to the three alternative destinations to Greece.

In [ ]:
origin_city = 'Manchester'
destinations = ['Oranjestad', 'Baku', 'Canberra']  

summary_stats = []

for city in destinations:
    try:
        print(f"\n Processing {origin_city} ➝ {city}")
        df = search_flights(origin_city, city)  # Using already written function

    
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
        df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')

        min_price = df['Price'].min()
        mean_price = df['Price'].mean()
        min_duration = df['Duration'].min()
        mean_duration = df['Duration'].mean()

        summary_stats.append({
            'Destination': city,
            'Min Price (£)': round(min_price, 2),
            'Avg Price (£)': round(mean_price, 2),
            'Min Duration (min)': int(min_duration),
            'Avg Duration (min)': round(mean_duration, 1)
        })

    except Exception as e:
        print(f" Could not process {city}: {e}")

summary_df = pd.DataFrame(summary_stats)

print(summary_df)

Since writing the above function, we had attempted to call the SkyScanner API again to test our code. However, due to the API call limit, we were unable to obtain this data again. Therefore, an image below from an old version of this notebook has been uploaded, showing the data obtained at the time of calling this function. In order to plot graphs from this data, a dataframe was manually created based on this image.

<img src='images/Average Flights DataFrame.png' />

In [ ]:
flight_stats = {
    "Destination": ['Oranjestad', 'Baku', 'Canberra'],
    "Min Price (£)": [423.32, 187.32, 1122.38],
    "Avg Price (£)": [629.51, 340.68, 1257.53],
    "Min Duration (min)": [840, 495, 1520],
    "Avg Duration (min)": [1299.4, 648.8, 1701.2]
}

flight_df = pd.DataFrame(flight_stats)

In [6]:
flight_df

,Destination,Min Price (£),Avg Price (£),Min Duration (min),Avg Duration (min)
0,Oranjestad,423.32,629.51,840,1299.4
1,Baku,187.32,340.68,495,648.8
2,Canberra,1122.38,1257.53,1520,1701.2
